In [11]:
import pandas
from bs4 import BeautifulSoup
import re
import nltk
import numpy
from gensim.models import word2vec

def clean_sentence( raw ):
    bs = BeautifulSoup(raw)
    letters_only = re.sub("[^a-zA-Z]"," ",bs.get_text())
    lower_case = letters_only.lower()
    words = lower_case.split()
    return words

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def review_to_sentences( review, tokenizer):
    #didn’t seem to work without it, thanks StackOverflow
    review = review.decode('utf-8')
    #strip out whitespace at beginning and end
    review = review.strip()
    raw_sentences = tokenizer.tokenize(review)
    sentences_list = []
    for sentence in raw_sentences:
        if len(sentence) > 0: #skip it if the sentence is empty
            cl_sent = clean_sentence(sentence)
            sentences_list.append(cl_sent)
    return sentences_list

data = pandas.read_csv("imdb_reviews.tsv", delimiter="\t")

#somehow need to make a sentences_for_all_reviews list
sentences_for_all_reviews = []

#clean_sentence(data)
for review in data['review']:
    sentences = review_to_sentences(review,tokenizer)
    sentences_for_all_reviews += sentences


num_attributes = 300 # Word vector dimensionality
min_word_count = 40 # Minimum word frequency
num_workers = 4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsample setting for frequent words
# Initialize and train the model (this will take some time)
model = word2vec.Word2Vec(sentences_for_all_reviews,workers=num_workers, size=num_attributes,min_count = min_word_count,window = context, sample = downsampling)
#saves memory if you’re done training it
model.init_sims(replace=True)

model.vocab
'chicago' in model.vocab
'iowa' in model.vocab
model.similarity('england','france')
model.similarity('england','paris')
model.most_similar('king')
model.most_similar('awful')
model.doesnt_match("man woman child kitchen".split())
model.doesnt_match("france england germany berlin".split())
model['king']
model['queen']
model['man']
model['woman']
(model['king'] - model['man'] + model['woman'])
model.most_similar(positive=['woman', 'king'], negative=['man'])

def makeAttributeVec(words, model, num_attributes):
    # Pre-initialize an empty numpy array (for speed)
    attributeVec = numpy.zeros((num_attributes,),dtype="float32")
    nwords = 0.
    # Loop over each word in the review and, if it is in the model’s
    # vocabulary, add its attribute vector to the total
    for word in words:
        if word in model.vocab:
            nwords = nwords + 1.
            attributeVec = numpy.add(attributeVec,model[word])
            
    # Divide the result by the number of words to get the average
    attributeVec = numpy.divide(attributeVec,nwords)
    return attributeVec


/Users/zacharyelkins/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/zacharyelkins/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.happierabroad.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
